In [12]:
import cv2 
import mediapipe as mp
import pandas as pd

In [ ]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)


df = pd.DataFrame({})
parts, x_coords, y_coords, z_coords, labels = [], [], [], [], []

# REMOVES THESE LANDMARKS (NOT THAT IMPORTANT)
exclude_landmarks = {
    mp_pose.PoseLandmark.NOSE,
    mp_pose.PoseLandmark.LEFT_EYE_INNER,
    mp_pose.PoseLandmark.LEFT_EYE,
    mp_pose.PoseLandmark.LEFT_EYE_OUTER,
    mp_pose.PoseLandmark.RIGHT_EYE_INNER,
    mp_pose.PoseLandmark.RIGHT_EYE,
    mp_pose.PoseLandmark.RIGHT_EYE_OUTER,
    mp_pose.PoseLandmark.LEFT_EAR,
    mp_pose.PoseLandmark.RIGHT_EAR,
    mp_pose.PoseLandmark.MOUTH_LEFT,
    mp_pose.PoseLandmark.MOUTH_RIGHT,
    mp_pose.PoseLandmark.LEFT_WRIST,
    mp_pose.PoseLandmark.RIGHT_WRIST,
    mp_pose.PoseLandmark.LEFT_THUMB,
    mp_pose.PoseLandmark.RIGHT_THUMB,
    mp_pose.PoseLandmark.LEFT_INDEX,
    mp_pose.PoseLandmark.RIGHT_INDEX,
    mp_pose.PoseLandmark.LEFT_PINKY,
    mp_pose.PoseLandmark.RIGHT_PINKY,
    mp_pose.PoseLandmark.LEFT_KNEE,
    mp_pose.PoseLandmark.RIGHT_KNEE,
    mp_pose.PoseLandmark.LEFT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_ANKLE,
    mp_pose.PoseLandmark.RIGHT_FOOT_INDEX,
    mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
    mp_pose.PoseLandmark.LEFT_HEEL,
    mp_pose.PoseLandmark.RIGHT_HEEL
}

with mp_pose.Pose() as pose:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        mp_drawing.draw_landmarks(
            frame,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS
        )

        # DRAWS THE SELECCTED BODY PARTS
        if results.pose_landmarks:
            #landmarks = results.pose_landmarks.landmark

            # visible = all(lm.visibility > 0.8 for id, lm in enumerate(landmarks) if mp_pose.PoseLandmark(id) not in exclude_landmarks)

            # #skip frame if not all are visible
            # if not visible:
            #     cv2.putText(frame, "Skipping: Not all parts visible", (20, 40),
            #                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            #     cv2.imshow('Pose Estimation', frame)
            #     continue
            
            h, w, _ = frame.shape
            for id, landmark in enumerate(results.pose_landmarks.landmark):
                landmark_enum = mp_pose.PoseLandmark(id)
                if landmark_enum in exclude_landmarks:
                    continue

                name = landmark_enum.name
                x, y = int(landmark.x * w), int(landmark.y * h)
                
                parts.append(name)
                x_coords.append(landmark.x)
                y_coords.append(landmark.y)
                z_coords.append(landmark.z)
                labels.append("ARMS RAISE")

                cv2.putText(frame, name.replace("LEFT_", "L_").replace("RIGHT_", "R_"), (x, y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1, cv2.LINE_AA)

        cv2.imshow('Pose Estimation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #df['id'] = id_
            df['parts'] = parts
            df['x_coords'] = x_coords
            df['y_coords'] = y_coords
            df['z_coords'] = z_coords
            df['labels'] = labels
            df.to_csv("landmarks.csv", index=False)
            break

cap.release()
cv2.destroyAllWindows()


F:\Github Projects\MediRehab-AI-V2\data\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [18]:
df_lm = pd.read_csv("landmarks.csv")
df_lm.head()

,parts,x_coords,y_coords,z_coords,labels
0,LEFT_SHOULDER,0.648593,0.743276,-0.088958,ARMS RAISE
1,RIGHT_SHOULDER,0.342293,0.753869,-0.190342,ARMS RAISE
2,LEFT_ELBOW,0.755681,0.997124,-0.121856,ARMS RAISE
3,RIGHT_ELBOW,0.245431,1.108702,-0.416588,ARMS RAISE
4,LEFT_HIP,0.591015,1.504511,-0.007863,ARMS RAISE


In [19]:
# df_parts_coords = df_lm.pivot(columns='parts', values=['x_coords', 'y_coords', 'z_coords'])
# df_parts_coords.head(n=20)
print(df_lm['parts'].value_counts())
#df_lm.loc[df_lm['parts'] == 'LEFT_SHOULDER', ['x_coords','y_coords','z_coords']].head()

parts
LEFT_SHOULDER     96
RIGHT_SHOULDER    96
LEFT_ELBOW        96
RIGHT_ELBOW       96
LEFT_HIP          96
RIGHT_HIP         96
Name: count, dtype: int64
